# Combines results across chromosomes

Post-run program to merge tables (summary and expanded table) across chromosomes to enable genome-wide analysis

Instructions:
1. Run each cell from the beginning in order
2. Modify the variables where suggested

In [7]:
import pandas as pd
import os
import glob
import re

In [10]:
from func_combine import *

# Individual

**MODIFY the following cell as needed**

In [6]:
# Let the notebook know what experiment type you would like to combine
# Specify it as a folder pattern with an asterisk * to match multiple directories
# Example: "splenic-B-cell_WT_insitu-hic_GSE82144_mm9_chr*_50Kb" will match all chromosomes
# of that specific experiment run
# folder_pattern = "splenic-B-cell_WT_insitu-hic_Kieffer-Kwon-2018_GSE82144_mm9_chr*_50Kb"
# folder_pattern = "DP-thymocytes_WT_hic_Guo-2022_GSE199059_mm10-remapped_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb"
# folder_pattern = "Repli-HiC_K562_WT_totalS_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-stripiness-1-p-enrich-0.4_chr*_50Kb"
# folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-normalized-2_chr*_50Kb"
folder_pattern = "GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr*_50Kb"
# folder_pattern = "Repli-HiC_K562_WT_totalS-stripiness-1_chr*_50Kb"
# folder_pattern = "GSE130275_mESC_WT_combined_2.6B_chr*_50Kb"

# exp_name = None
exp_name = "GSE199059_CD69negDPWTR1R2R3R4_merged_chr*_50Kb" 
# exp_name = "Repli-HiC_K562_WT_totalS_chr*_50Kb"

# Location of the results folders
# Where is the folder above located?
results_dir = "/nfs/turbo/umms-minjilab/sionkim/miajet_output/"

# This is the suffix after the "_results" in each folder 
# You can specify "all" to combine all results or a specifically thresholded output
# Examples: "all", "p-0.01", "saliency-90", "saliency-90-p-0.1"
result_type = ["saliency-90-p-0.1", "saliency-90-p-0.05", "saliency-90-p-0.01", "saliency-90", "p-0.1", "p-0.05", "p-0.01", "all"]

In [7]:
if isinstance(result_type, str):
    df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, result_type)
    # Will print out the parametesr of the experiment
    # So that you can confirm this is indeed the correct experiment you are combining

    df_summary = [df_s]
    df_expanded = [df_e]
    df_bedpe = [df_b]
else:
    df_summary = []
    df_expanded = []
    df_bedpe = []
    for rt in result_type:
        df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, rt, exp_name=exp_name)
        df_summary.append(df_s)
        df_expanded.append(df_e)
        df_bedpe.append(df_b)

GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr _50Kb
GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb
['/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr4_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr3_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr15_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr6_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr9_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chrX_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr16_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr1_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_C

/tmp/ipykernel_272653/1550320764.py:128: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_summary = pd.concat(summary_frames,  ignore_index=True)
/tmp/ipykernel_272653/1550320764.py:129: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the relevant entries before the concat operation.
  combined_expanded = pd.concat(expanded_frames, ignore_index=True)


GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr _50Kb
GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb
['/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr4_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr3_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr15_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr6_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr9_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chrX_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr16_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr1_50Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_C

In [6]:
# Can visually inspect the summary table
df_summary[0]

,unique_id,chrom,start,end,length,input_mean,angle_mean,width_mean,jet_saliency,ks,p-val_raw,p-val_corr,stripiness
0,chr1_28_9,chr1,1.739396e+08,1.743253e+08,900000,0.132,110.422,7.109,0.054,1.000,0.000,0.000,0.041
1,chr1_64_10,chr1,7.121808e+07,7.159345e+07,1000000,0.121,109.471,4.348,0.051,0.450,0.017,0.054,0.025
2,chr1_1541_10,chr1,1.739067e+08,1.742917e+08,800000,0.134,113.905,7.393,0.041,0.938,0.000,0.000,0.035
3,chr1_419_11,chr1,1.020671e+08,1.026250e+08,850000,0.080,88.292,7.144,0.047,1.000,0.000,0.000,0.014
4,chr1_20_16,chr1,1.857944e+08,1.867769e+08,1250000,0.132,78.523,8.578,0.075,0.760,0.000,0.000,0.039
...,...,...,...,...,...,...,...,...,...,...,...,...,...
157,chrX_105_14,chrX,6.600922e+07,6.685632e+07,3200000,0.012,119.451,9.802,0.008,0.219,0.047,0.079,0.002
158,chrX_1044_4,chrX,1.712626e+07,1.737590e+07,400000,0.012,90.720,2.755,0.006,1.000,0.000,0.001,0.002
159,chrX_4016_4,chrX,1.063709e+08,1.066141e+08,450000,0.015,89.417,3.999,0.006,0.667,0.017,0.042,0.003
160,chrX_119_9,chrX,3.953824e+07,3.976459e+07,400000,0.012,98.998,4.649,0.005,0.875,0.001,0.008,0.001


In [7]:
# Can visually inspect the expanded table
df_expanded[0]

,unique_id,chrom,x (bp),y (bp),x (pixels),y (pixels),width,angle_imagej,ridge_strength
0,chr1_20_16,chr1,1.865853e+08,1.848068e+08,5252.691,58.993,9.163,90.723,0.010
1,chr1_20_16,chr1,1.866257e+08,1.847748e+08,5252.810,57.970,9.119,91.995,0.010
2,chr1_20_16,chr1,1.866675e+08,1.847436e+08,5252.961,56.937,8.761,93.373,0.009
3,chr1_20_16,chr1,1.867117e+08,1.847136e+08,5253.161,55.888,8.042,94.894,0.009
4,chr1_20_16,chr1,1.867377e+08,1.846681e+08,5252.885,54.877,8.277,96.592,0.008
...,...,...,...,...,...,...,...,...,...
2033,chrX_13_8,chrX,1.886369e+07,1.848282e+07,528.574,78.759,3.567,90.397,0.001
2034,chrX_13_8,chrX,1.889771e+07,1.844674e+07,528.545,77.768,3.567,89.579,0.001
2035,chrX_105_14,chrX,6.681408e+07,6.434742e+07,1855.318,49.262,11.877,125.818,0.001
2036,chrX_105_14,chrX,6.668986e+07,6.484107e+07,1860.542,58.000,11.941,117.658,0.001


In [8]:
# Can visually inspect the bedpe table
df_bedpe[0]

,chrom1,start1,end1,chrom2,start2,end2
0,chr1,43036676,43061676,chr1,44117732,44142732
1,chr1,43074360,43099360,chr1,44084735,44109735
2,chr1,43108486,43133486,chr1,44048054,44073054
3,chr1,43140552,43165552,chr1,44010325,44035325
4,chr1,43158386,43183386,chr1,43967960,43992960
...,...,...,...,...,...,...
2033,chrX,155950983,155975983,chrX,156334898,156359898
2034,chrX,155992574,156017574,chrX,156307032,156332032
2035,chrX,156033195,156058195,chrX,156277527,156302527
2036,chrX,156072731,156097731,chrX,156246313,156271313


**(OPTIONAL) modify as needed**

In [8]:
save_name = folder_pattern.replace('*', f'_combined') 
# This is the save name for the combined results

save_dir = os.path.join(results_dir, save_name)
# Folder to save the combined results (summary and expanded tables)
# You may overwrite the `results_dir` if you wish to save to a different location

print("Saving combined tables to:")
print(f"* Folder location: {save_dir}")

save_dir_summary = []
save_dir_expanded = []
save_dir_bedpe = []
for rt in result_type:
    print(f"* Result type: {rt}")
    save_name_file = folder_pattern.replace('*', f'_combined_{rt}')

    # Below is the code to print out the save locations to help you modify the variables abovef
    # Once satisfied, move on to the next cell
    save_dir_s = os.path.join(save_dir, f"{save_name_file}_summary_table.csv")
    save_dir_e = os.path.join(save_dir, f"{save_name_file}_expanded_table.csv")
    save_dir_b = os.path.join(save_dir, f"{save_name_file}_juicer-visualize.bedpe")

    save_dir_summary.append(save_dir_s)
    save_dir_expanded.append(save_dir_e)
    save_dir_bedpe.append(save_dir_b)

    print(f"\t* Summary table: {save_name_file}_summary_table.csv")
    print(f"\t* Expanded table: {save_name_file}_expanded_table.csv")
    print(f"\t* Bedpe table: {save_name_file}_juicer-visualize.bedpe")

Saving combined tables to:
* Folder location: /nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb
* Result type: saliency-90-p-0.1
	* Summary table: GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.1_50Kb_summary_table.csv
	* Expanded table: GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.1_50Kb_expanded_table.csv
	* Bedpe table: GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.1_50Kb_juicer-visualize.bedpe
* Result type: saliency-90-p-0.05
	* Summary table: GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.05_50Kb_summary_table.csv
	* Expanded table: GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.05_50Kb_expanded_table.csv
	* Bedpe table: GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.05_50Kb_juicer-visualize.bedpe
* Result type: saliency-90-p-0.01
	* Summary table: GSE199059_CD69negDPWTR1R

In [9]:
# Make directory if it doesn't exist already and save the files
os.makedirs(save_dir, exist_ok=True)

for i in range(len(result_type)):
    save_csv(df_summary[i], save_dir_summary[i], parameter_str)
    save_csv(df_expanded[i], save_dir_expanded[i], parameter_str)
    df_bedpe[i].to_csv(save_dir_bedpe[i], sep='\t', index=False, header=False) 
    print(f"Saved\n *{save_dir_summary[i]}\n *{save_dir_expanded[i]}\n *{save_dir_bedpe[i]}")

Saved
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.1_50Kb_summary_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.1_50Kb_expanded_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.1_50Kb_juicer-visualize.bedpe
Saved
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_saliency-90-p-0.05_50Kb_summary_table.csv
 */nfs/turbo/umms-minjilab/sionkim/miajet_output/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_chr_combined_50Kb/GSE199059_CD69negDPWTR1R2R3R4_merged-root2_

# Multiple samples


Loads in the `submit_all_config.yaml` used to submit batch jobs on Slurm to now combine the results

In [4]:
import yaml

import sys
sys.path.append('..')
from utils.plotting import genomic_labels

In [5]:
with open("../submit_all_config.yaml") as cf:
    config = yaml.safe_load(cf)

samples = config["samples"]

results_dir = "/nfs/turbo/umms-minjilab/sionkim/miajet_output/"
result_types = [
    "saliency-90-p-0.1", "saliency-90-p-0.05", "saliency-90-p-0.01",
    "saliency-90", "p-0.1", "p-0.05", "p-0.01", "all"
]

for key, attributes in samples.items():
    print(f"Processing experiment: {key}")

    f = attributes["file"]
    basename = os.path.basename(f)
    exp = os.path.splitext(basename)[0]  # Remove file extension

    resolution = genomic_labels(int(attributes["res"]))

    folder_pattern = f"{exp}_chr*_{resolution}"
    save_name = folder_pattern.replace('*', '_combined')
    save_dir = os.path.join(results_dir, save_name)
    os.makedirs(save_dir, exist_ok=True)

    # Combine results for each result type
    df_summary_list = []
    df_expanded_list = []
    df_bedpe_list   = []
    for rt in result_types:
        print(f"\tCombining result type: {rt}")
        df_s, df_e, df_b, parameter_str = combine_results(results_dir, folder_pattern, rt)
        df_summary_list.append(df_s)
        df_expanded_list.append(df_e)
        df_bedpe_list.append(df_b)

        # Construct file paths
        save_base = folder_pattern.replace('*', f"_combined_{rt}")
        summary_fp = os.path.join(save_dir, f"{save_base}_summary_table.csv")
        expanded_fp = os.path.join(save_dir, f"{save_base}_expanded_table.csv")
        bedpe_fp = os.path.join(save_dir, f"{save_base}_juicer-visualize.bedpe")

        # Save outputs
        save_csv(df_s, summary_fp, parameter_str)
        save_csv(df_e, expanded_fp, parameter_str)
        df_b.to_csv(bedpe_fp, sep='\t', index=False, header=False)

        print(f"\tSaved summary:  {summary_fp}")
        print(f"\tSaved expanded: {expanded_fp}")
        print(f"\tSaved BEDPE:    {bedpe_fp}")

    print()  


Processing experiment: c_elegans_CA1200_L2_L3_JK07_JK08_control_auxin_1hr_hic_Kim-2023_GSE188849_ce10
	Combining result type: saliency-90-p-0.1
c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10_chr _5Kb
c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10_chr_combined_5Kb
['/nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10_chrIV_5Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10_chrV_5Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10_chr_combined_5Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-2023_GSE188849_ce10_chrX_5Kb', '/nfs/turbo/umms-minjilab/sionkim/miajet_output/c-elegans-CA1200-L2-L3-JK07-JK08_control-auxin-1hr_hic_Kim-202